<a href="https://colab.research.google.com/github/esvm/IF704-chatbot/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Instructions

1. Create an account in [Kaggle](https://www.kaggle.com/)
2. Go to your account (https://www.kaggle.com/<yourusername>/account)
3. Generate a new API Token if you don't have one
4. Upload the downloaded `kaggle.json` in this notebook folder

# Installing [Kaggle](https://www.kaggle.com/)

In [10]:
! pip install kaggle

In [12]:
import os

if not os.path.exists('~/.kaggle'):
        os.makedirs('~/.kaggle')

In [16]:
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

# Downloading Dataset

In [17]:
! kaggle datasets download -d rajathmc/cornell-moviedialog-corpus

 52% 5.00M/9.58M [00:00<00:00, 11.1MB/s]
100% 9.58M/9.58M [00:00<00:00, 19.5MB/s]


In [19]:
if not os.path.exists('./dataset'):
        os.makedirs('./dataset')

In [20]:
! mv cornell-moviedialog-corpus.zip ./dataset

In [26]:
import zipfile
with zipfile.ZipFile('./dataset/cornell-moviedialog-corpus.zip', 'r') as zip_ref:
    zip_ref.extractall('./dataset')

# Setup Dependencies

# Reading Data